In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import joblib

# 📌 1. Cargar los datos
file_path = "../data/final/sri_autos_features.csv"  # Ajusta la ruta
#  Cargar el dataset procesado
df = pd.read_csv(file_path, delimiter=';', encoding="latin-1", low_memory=False)
print(df.columns)

# 📌 2. Seleccionar las columnas relevantes
columnas_numericas = ["year_modelo", "cilindraje", "avaluo"]
columnas_categoricas = ["marca", "modelo", "pais", "clase", "sub_clase", "tipo_combustible"]

# 📌 3. Aplicar One-Hot Encoding a variables categóricas
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_data = encoder.fit_transform(df[columnas_categoricas])

# 📌 4. Convertir One-Hot Encoding a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columnas_categoricas))

# 📌 5. Concatenar variables numéricas con las categóricas codificadas
df_final = pd.concat([encoded_df, df[columnas_numericas]], axis=1)

# 📌 6. Normalizar variables numéricas
scaler = StandardScaler()
df_final[columnas_numericas] = scaler.fit_transform(df_final[columnas_numericas])

# 📌 7. Separar variables de entrada (X) y salida (y)
X = df_final.drop(columns=["avaluo"])
y = df_final["avaluo"]

# 📌 8. Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 9. Construir la red neuronal MLP
modelo_mlp = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1)  # Predicción del precio
])

# 📌 10. Compilar el modelo
modelo_mlp.compile(optimizer="adam", loss="mse", metrics=["mae"])

# 📌 11. Entrenar el modelo
modelo_mlp.fit(X_train, y_train, epochs=35, batch_size=32, validation_data=(X_test, y_test))

# 📌 12. Guardar el modelo y los preprocesadores
modelo_mlp.save("../models/mlp_model.h5")
joblib.dump(encoder, "../models/mlp_encoder.pkl")
joblib.dump(scaler, "../models/mlp_scaler.pkl")

print("✅ Modelo MLP entrenado y guardado correctamente.")

Index(['marca', 'modelo', 'pais', 'year_modelo', 'clase', 'sub_clase',
       'avaluo', 'cilindraje', 'tipo_combustible'],
      dtype='object')


c:\Users\SEBPE\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.5286 - mae: 0.5064 - val_loss: 0.2180 - val_mae: 0.3111
Epoch 2/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1643 - mae: 0.2638 - val_loss: 0.1877 - val_mae: 0.2762
Epoch 3/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1132 - mae: 0.2163 - val_loss: 0.1941 - val_mae: 0.2758
Epoch 4/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0868 - mae: 0.1916 - val_loss: 0.1844 - val_mae: 0.2641
Epoch 5/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0814 - mae: 0.1853 - val_loss: 0.1824 - val_mae: 0.2658
Epoch 6/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0735 - mae: 0.1720 - val_loss: 0.1840 - val_mae: 0.2734
Epoch 7/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0699 - mae: 0.1676 - val_loss: 0.1815 - val_mae: 0.2610
Epoch 8/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0668 - mae: 0.1641 - val_loss: 0.1828 - val_mae: 0.2611
Epoch 9/35
306/306 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - lo

✅ Modelo MLP entrenado y guardado correctamente.


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 📌 1. Predecir en el conjunto de prueba
y_pred = modelo_mlp.predict(X_test)

# 📌 2. Evaluar el rendimiento del modelo
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n📊 Evaluación del Modelo MLP:")
print(f"✅ MAE (Error Absoluto Medio): {mae:.2f}")
print(f"✅ RMSE (Raíz del Error Cuadrático Medio): {rmse:.2f}")
print(f"✅ R² Score: {r2:.4f}")

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

📊 Evaluación del Modelo MLP:
✅ MAE (Error Absoluto Medio): 0.26
✅ RMSE (Raíz del Error Cuadrático Medio): 0.42
✅ R² Score: 0.8189


In [ ]:
import tensorflow as tf
import pandas as pd
import joblib
import numpy as np

# 📌 Cargar el modelo y preprocesadores con manejo de errores
try:
    modelo_mlp = tf.keras.models.load_model(
        "../models/mlp_model.h5",
        custom_objects={"mse": tf.keras.losses.MeanSquaredError()}  # Solución al error de métricas
    )
    encoder = joblib.load("../models/mlp_encoder.pkl")
    scaler = joblib.load("../models/mlp_scaler.pkl")
    print("✅ Modelos y preprocesadores cargados correctamente.")
except Exception as e:
    print(f"❌ Error al cargar los modelos/preprocesadores: {e}")
    exit()

# 📌 Obtener las columnas originales del `encoder`
try:
    encoder_columns_original = encoder.feature_names_in_
    print(f"🔍 Columnas originales del encoder: {encoder_columns_original}")
except Exception as e:
    print(f"❌ Error al obtener las columnas originales del encoder: {e}")
    exit()

# 📌 Definir un nuevo auto para predecir su precio
nuevo_auto = {
    "modelo": "TESLA MODEL Y",
    "marca": "TESLA",
    "pais": "ESTADOS UNIDOS",
    "year_modelo": 2023,
    "clase": "AUTOMOVIL",
    "sub_clase": "ELECTRICO",
    "cilindraje": 0,
    "tipo_combustible": "ELECTRICO"
}

# 📌 Convertir a DataFrame
nuevo_auto_df = pd.DataFrame([nuevo_auto])

# 📌 Usar las columnas categóricas originales del encoder
columnas_categoricas = encoder_columns_original.tolist()
columnas_numericas = ["year_modelo", "cilindraje"]  # Solo las numéricas usadas en `scaler`

# 📌 Verificar que todas las columnas categóricas existen en `nuevo_auto_df`
for col in columnas_categoricas:
    if col not in nuevo_auto_df.columns:
        print(f"⚠️ Advertencia: La columna categórica '{col}' no está en el DataFrame.")
        exit()

# 📌 Aplicar One-Hot Encoding con manejo de errores
try:
    encoded_nuevo_auto = encoder.transform(nuevo_auto_df[columnas_categoricas])
    encoded_nuevo_auto_df = pd.DataFrame(encoded_nuevo_auto, columns=encoder.get_feature_names_out())
    print("✅ One-Hot Encoding aplicado correctamente.")
except Exception as e:
    print(f"❌ Error en One-Hot Encoding: {e}")
    exit()

# 📌 Verificar que las columnas numéricas existen en `nuevo_auto_df`
for col in columnas_numericas:
    if col not in nuevo_auto_df.columns:
        print(f"⚠️ Advertencia: La columna numérica '{col}' no está en el DataFrame.")
        exit()

# 📌 Asegurar que `scaler` no use `avaluo`
try:
    scaler_columns = scaler.feature_names_in_  # Verificamos con qué columnas se entrenó el `scaler`
    columnas_validas_scaler = [col for col in columnas_numericas if col in scaler_columns]
    nuevo_auto_df[columnas_validas_scaler] = scaler.transform(nuevo_auto_df[columnas_validas_scaler])
    print("✅ Normalización aplicada correctamente.")
except Exception as e:
    print(f"❌ Error al normalizar datos: {e}")
    exit()

# 📌 Concatenar datos codificados y normalizados con verificación de tamaño
try:
    nuevo_auto_final = pd.concat([encoded_nuevo_auto_df, nuevo_auto_df[columnas_validas_scaler]], axis=1)
    print(f"✅ Datos preparados con éxito. Dimensión final: {nuevo_auto_final.shape}")
except Exception as e:
    print(f"❌ Error al concatenar datos: {e}")
    exit()

# 📌 Verificar si la dimensión de `nuevo_auto_final` coincide con la entrada del modelo
expected_features = modelo_mlp.input_shape[1]
if nuevo_auto_final.shape[1] != expected_features:
    print(f"❌ Error: Dimensión incorrecta. Se esperaban {expected_features} columnas, pero se obtuvieron {nuevo_auto_final.shape[1]}.")
    exit()

# 📌 Asegurar la forma correcta de la entrada
entrada_modelo = nuevo_auto_final.to_numpy().reshape(1, -1)

# 📌 Predecir el precio del auto
try:
    predicted_price = modelo_mlp.predict(entrada_modelo)[0][0]
    print(f"\n✅ Precio Predicho para el Auto: ${predicted_price:.2f}")
except Exception as e:
    print(f"❌ Error al realizar la predicción: {e}")


✅ Modelos y preprocesadores cargados correctamente.
🔍 Columnas originales del encoder: ['marca' 'modelo' 'pais' 'clase' 'sub_clase' 'tipo_combustible']
✅ One-Hot Encoding aplicado correctamente.
❌ Error al normalizar datos: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- avaluo

✅ Datos preparados con éxito. Dimensión final: (1, 3498)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step

✅ Precio Predicho para el Auto: $595.73


: 